Simple test showing use of pyembmod

In [1]:
import os.path
import sys
import os
import numpy as np

import argparse
modpaths = os.environ.get('PYBERTHA_MOD_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)

sys.path.insert(0, "../common")
modpaths = os.environ.get('COMMON_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)


In [2]:
import fde_utils
from fde_utils import embedoption
#import pyembmod
from pyembmod import GridDensityFactory



In [3]:
pyembopt = embedoption

pyembopt.gridfname = 'grid_adf.dat'

pyembopt.debug = True
#pyembopt.linemb = False
#pyembopt.nofde = False
#pyembopt.verbosity = args.verbosity
#pyembopt.thresh = args.thresh
#pyembopt.static_field = 0.00
#pyembopt.fmax = None
#pyembopt.fdir = None
pyembopt.tot_charge = 0
pyembopt.core_charge = 0
pyembopt.activefile = 'H2O.xyz'
pyembopt.envirofile = 'NH3.xyz'
pyembopt.gtype = 2
pyembopt.jobtype = 'psi4'
pyembopt.thresh_conv = 1.0e-8
pyembopt.basis = 'cc-pvtz'  #<- refers to the (FDE) environment 
pyembopt.excfuncenv = 'blyp'

gparam = "50,230".split(",")

if pyembopt.jobtype == 'adf':
  gparam = [float(m) for m in gparam]
  if not isinstance(gparam[0],float):
     raise TypeError("adf grid(param) accuracy must be float")
  pyembopt.param = gparam[0]
else:
  gparam = [int(m) for m in gparam]
  pyembopt.param = tuple(gparam)


In [4]:
# psi4 minimal set up
import psi4
from molecule import Molecule
mol_act = Molecule(pyembopt.activefile)
mol_act.finalize()
psi4.core.set_output_file('psi.out', False)
act_obs =  'cc-pvdz'
molobj=psi4.geometry(mol_act.geometry())
psi4.set_options({'basis': act_obs,
                      'puream': 'True',
                      'DF_SCF_GUESS': 'False',
                      'scf_type': 'DIRECT',
                      'dft_radial_scheme' : 'becke',
                      #'dft_radial_points': 49,
                      'dft_spherical_points' : 434,
                      'cubeprop_tasks': ['orbitals'],
                      'e_convergence': 1e-8,
                      'd_convergence': 1e-8})
ene,wfn = psi4.energy('b3lyp',return_wfn=True)
Cocc = np.array(wfn.Ca_subset('AO', 'OCC'))
embed = fde_utils.emb_wrapper(molobj,pyembopt,act_obs)
# here we need the active system density expressed on the grid

embed.set_density(Cocc)
nel_ACT =embed.rho_integral()
# use GridDensityFactory
#grid = embed.grid()
#activesys = GridDensityFactory(molobj,grid,act_obs)

#if isinstance(Cocc,np.ndarray):
#      rho = activesys.from_Cocc(Cocc)
#elif Dmat is not None: #also ovap has to be != None
#      rho = activesys.from_D(Dmat,self.__ovap)
#check number of electron
#nel_ACT = np.matmul(rho,grid[:,3])
print("n. el act sys: %.8f\n" % (nel_ACT))
res = embed.make_embpot()




(50, 230)
embfactory Options:
pyemb job type : psi4, grid type : 2, functional (enviro) : blyp, e/d thresh : 1.00e-08
grid specs (accuracy / radial & spherical points)
(50, 230)
      N    -1.395591   -0.021564    0.000037
      H    -1.629811    0.961096   -0.106224
      H    -1.862767   -0.512544   -0.755974
      H    -1.833547   -0.330770    0.862307
0 1
symmetry c1
no_com
no_reorient

PHI dim: 69124,72

N. basis functions: 72

N. grid points 69124

env num el. 9.99999

n. el act sys: 10.09603402



In [5]:
print("chek Vemb dim")
print(res.shape)

chek Vemb dim
(24, 24)


In [6]:
wfn.basisset().nbf()

24